In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
import os
import gc
!pip install segmentation-models
!pip install git+https://github.com/qubvel/segmentation_models

In [ ]:
trainImgPath = "/kaggle/input/severstal-steel-defect-detection/train_images/"
trainCsv = "/kaggle/input/severstal-steel-defect-detection/train.csv"
data=pd.read_csv(trainCsv)
data.ClassId=data.ClassId.astype(int)

In [ ]:
WIDTH=192
HEIGHT=192
TRAINING_SIZE=12568
NUM_OF_CLASSES=5

In [ ]:
def convert_to_mask(encoded_pixels, class_id=0):
    counts=[]
    mask=np.zeros((256*1600), dtype=np.uint8) #don't change this
    if(class_id==0):
        mask=np.reshape(mask,(1600,256)) #don't change this
        mask=cv2.resize(mask,(HEIGHT,WIDTH)).T
        return mask
    pre_mask=np.asarray([int(point) for point in encoded_pixels.split()])
    for index,count in enumerate(pre_mask):
        if(index%2!=0):
            counts.append(count)
    i=0
    len_count=len(counts)
    for index,pixel in enumerate(pre_mask):
        if(index%2==len_count):
            if(i==l):
                break
            mask[pixel:pixel+counts[i]]=1
            i+=1
    mask=np.reshape(mask,(1600,256)) #don't change this
    mask=cv2.resize(mask,(HEIGHT,WIDTH)).T
    return mask

In [ ]:
def generate_imgs():
    list_of_imgs=np.empty((TRAINING_SIZE,HEIGHT,WIDTH,3), dtype=np.uint8)
    list_of_masks=np.empty((TRAINING_SIZE,HEIGHT,WIDTH,NUM_OF_CLASSES), dtype=np.uint8)
    i=0
    encoder=np.zeros(NUM_OF_CLASSES)
    imgs=os.listdir(trainImgPath)
    for img in imgs:
        mask=convert_to_mask(0,0)
        imgpath = trainImgPath + '/' + img
        imgs_in_csv=data.loc[data['ImageId'] == img]
        if(len(imgs_in_csv)):
            for index,row in imgs_in_csv.iterrows():
                rle=row["EncodedPixels"]
                cls=row["ClassId"]
                if(cls!=0):
                    encoder[cls]=1
                    mask1=convert_to_mask(rle,1)
                    mask=np.logical_or(mask1,mask)*1
                else:   
                    encoder[0]=0
                    mask1=convert_to_mask(rle)
                    mask=np.logical_or(mask1,mask)*1
        else:
            encoder[0]=0
        y=np.zeros((WIDTH,HEIGHT,NUM_OF_CLASSES))
        y[...,0]=mask
        y[0,...]=encoder
        list_of_masks[i]=y
        img=cv2.imread(imgpath)
        img=cv2.resize(img,(HEIGHT,WIDTH))
        list_of_imgs[i]=img
        i+=1
        del y
        del mask
        del img
        gc.collect()
        if(i%1000==0):
            print("1000 imgs have been loaded")
    return list_of_imgs,list_of_masks
    

In [ ]:
x_train,y_train=generate_imgs()

In [ ]:
print(x_train.shape,y_train.shape)

In [ ]:
from keras import backend as K
def dice_coef(y_true, y_pred, smooth=1):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

In [ ]:
from segmentation_models import Unet
from segmentation_models import get_preprocessing
from segmentation_models.losses import bce_jaccard_loss
from segmentation_models.metrics import iou_score
 
network = 'resnet34'
process_input = get_preprocessing(network)
x_train = process_input(x_train)
model = Unet(network,input_shape = (192, 192, 3),classes=NUM_OF_CLASSES,activation='sigmoid')
model.compile('adam', loss='binary_crossentropy',metrics=[dice_coef])


In [ ]:
model.fit(
    x=x_train,
    y=y_train,
    batch_size=16,
    epochs=10, 
)